In [1]:
import requests as req
import pandas as pd
import sqlite3
from time import sleep

In [2]:
ENDPOINT = 'https://api.hh.ru/vacancies'
URL_HEADERS = {'User-Agent': 'Mozilla/5.0'}
DB_NAME = 'hh.db'
TABLE_NAME = 'vacancies_api'
LIMIT_OF_VACANCY_DETAIL = 1000

In [3]:

api_params = {'text':'python middle developer',
              'search_field':'name',
              'per_page': 10,
              'page':0}
vacancyDetailInfo =[]

In [4]:
def parsJsonAppendData (data):
    for result in data['items']:
        vacancyRow = []
        listOfRequirements = []
        vacancyName = result['name']
        companyName = result['employer']['name']
        vacancyid = result['id']
        vacancyDetail = req.get(ENDPOINT+'/'+vacancyid, headers = URL_HEADERS).json()
        print(ENDPOINT+'/'+vacancyid)
        vacancyDescription = vacancyDetail['description']
        for skill in vacancyDetail['key_skills']:
            listOfRequirements.append(skill['name'])
        stringOfRequirements = ', '.join(listOfRequirements)
        vacancyRow.append(companyName)
        vacancyRow.append(vacancyName)
        vacancyRow.append(vacancyDescription)
        vacancyRow.append(stringOfRequirements)
        vacancyDetailInfo.append(vacancyRow)

In [5]:
while (True):
    results = req.get(ENDPOINT, headers = URL_HEADERS, params = api_params).json()
    if (len(results['items']) == 0 or len(vacancyDetailInfo)>=LIMIT_OF_VACANCY_DETAIL):
        break
    parsJsonAppendData(results)
    print(f'Список детальной информации о вакансиях содержит {len(vacancyDetailInfo)} записей')
    api_params['page'] += 1
    sleep(10)
    

https://api.hh.ru/vacancies/82841724
https://api.hh.ru/vacancies/83300859
https://api.hh.ru/vacancies/83438237
https://api.hh.ru/vacancies/81949300
https://api.hh.ru/vacancies/83409394
https://api.hh.ru/vacancies/82858718
https://api.hh.ru/vacancies/82825127
https://api.hh.ru/vacancies/83234434
https://api.hh.ru/vacancies/83409596
https://api.hh.ru/vacancies/82987200
Список детальной информации о вакансиях содержит 10 записей
https://api.hh.ru/vacancies/83279670
https://api.hh.ru/vacancies/83225173
https://api.hh.ru/vacancies/82402579
https://api.hh.ru/vacancies/83345919
https://api.hh.ru/vacancies/82119961
https://api.hh.ru/vacancies/81941871
https://api.hh.ru/vacancies/83409606
https://api.hh.ru/vacancies/82870104
https://api.hh.ru/vacancies/83256342
https://api.hh.ru/vacancies/83279671
Список детальной информации о вакансиях содержит 20 записей
https://api.hh.ru/vacancies/82055830
https://api.hh.ru/vacancies/82936068
https://api.hh.ru/vacancies/82910400
https://api.hh.ru/vacancies/8

In [9]:
for line in vacancyDetailInfo:
    if 

['Webtronics', 'Junior/Middle backend разработчик на Fast API (Python)', '<p>Привет! Мы Webtronics. Ищем в команду FastAPI (Python) разработчика в идеале на фуллтайм, но возможен и проектный подход или решение точечных задач по почасовой оплате. Подробнее о том что мы делаем и что предстоит тебе можно узнать тут https://webtronics.ru/app. Код храним в Gitlab. Задачи ставим в JIRA.</p> <p><strong>Обязанности:</strong></p> <ul> <li> <p>Проработка архитектурных решений Backend для web-сервисов и мобильных приложений заказчиков;</p> </li> <li> <p>Прототипирование backend для web-сервисов и мобильных приложений заказчиков;</p> </li> <li> <p>Разработка backend для web-сервисов и мобильных приложений заказчиков;</p> </li> <li> <p>Анализ требований и выработка технических решений;</p> </li> <li> <p>Оценка ресурсов и сроков выполнения задач в рамках своего участка работ;</p> </li> <li> <p>Участие в разработке архитектуры Backend web-сервисов и мобильных приложений и проектировании баз данных, а

In [8]:
results

{'items': [],
 'found': 194,
 'pages': 20,
 'per_page': 10,
 'page': 20,
 'clusters': None,
 'arguments': None,
 'alternate_url': 'https://hh.ru/search/vacancy?enable_snippets=true&items_on_page=10&page=20&search_field=name&text=python+middle+developer'}

In [6]:
table_columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=table_columns,index=None)

In [8]:
connection = sqlite3.connect(DB_NAME)
df.to_sql(TABLE_NAME,connection,if_exists='replace',index=False)
connection.close()